Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots,ApproxFun

In [2]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,[0,T])
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

Φ = A\eye(n);

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

1.6653345369377348e-16 - 0.2683546905319051im
1.4468677991819057e-16 + 0.26835469053190514im
0.03747531973228922 + 1.0000000000000007im
-0.03747531973228926 + 0.9999999999999999im

Floquet Multipliers = 

0.6651802570066614 - 0.7466828146465894im
0.6651802570066613 + 0.7466828146465895im
-1.1249427991479368 - 2.3601090341805867e-15im
-0.8889340869219554 + 5.036290646901528e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

208

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,[0,10T])
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,43.5083 35.4377,43.4831 35.4471,43.4329 35.4611,43.3577 35.4797,43.2576 35.5031,43.1327 35.5311,42.9832 35.5637,42.8094 35.601,42.6114 35.643,42.3897 
 35.6897,42.1445 35.741,41.8761 35.797,41.585 35.8576,41.2717 35.9229,40.9365 35.9928,40.5801 36.0674,40.2029 36.1467,39.8056 36.2306,39.3888 36.3192,38.9532 
 36.4124,38.4994 36.5103,38.0282 36.6128,37.5404 36.72,37.0368 36.8318,36.5183 36.9482,35.9858 37.0693,35.4402 37.1951,34.8825 37.3254,34.3137 37.4604,33.7349 
 37.6001,33.1472 37.7443,32.5517 37.8932,31.9496 38.0467,31.342 38.2049,30.7304 38.3677,30.1158 38.535,29.4998 38.707,28.8835 38.8836,28.2686 39.0649,27.6563 
 39.2507,27.0481 39.4411,26.4456 39.6361,25.8503 39.8357,25.2639 40.04,24.6877 40.2488,24.1237 40.4621,23.5733 40.6801,23.0383 40.9027,22.5204 41.1298,22.0213 
 41.3615,21.5429 41.5977,21.0869 41.8386,20.6551 42.084,20.2494 42.3339,19.8716 42.5884,19.5235 42.8474,19.2071 43.111,18.9241 43.3791,18.6766 43.6518,18.4663 
 43.929,18.2952 44.2107,18.1651 44.4969,18.0779 44.7877,18.0356 45.0829,18.0399 45.3827,18.0927 45.687,18.1959 45.9957,18.3513 46.309,18.5606 46.6267,18.8256 
 46.9489,19.1481 47.2756,19.5298 47.6067,19.9723 47.9424,20.4774 48.2824,21.0465 48.627,21.6813 48.976,22.3832 49.3294,23.1538 49.6872,23.9945 50.0495,24.9066 
 50.4162,25.8915 50.7873,26.9503 51.1629,28.0844 51.5428,29.2948 51.9271,30.5827 52.3159,31.9491 52.709,33.395 53.1065,34.9213 53.5083,36.5288 53.9145,38.2183 
 54.3251,39.9905 54.7401,41.8462 55.1594,43.7859 55.583,45.8103 56.0109,47.9197 56.4432,50.1146 56.8798,52.3954 57.3207,54.7625 57.7659,57.2161 58.2154,59.7564 
 58.6692,62.3835 59.1272,65.0977 59.5896,67.8989 60.0562,70.7872 60.527,73.7626 61.0021,76.8249 61.4815,79.974 61.965,83.2098 62.4528,86.5319 62.9449,89.9402 
 63.4411,93.4342 63.9415,97.0136 64.4461,100.678 64.9549,104.427 65.4679,108.259 65.9851,112.174 66.5064,116.172 67.0318,120.251 67.5614,124.411 68.0951,128.65 
 68.633,132.967 69.1749,137.362 69.721,141.831 70.2711,146.375 70.8254,150.99 71.3837,155.675 71.9461,160.429 72.5125,165.248 73.083,170.129 73.6575,175.071 
 74.2361,180.069 74.8187,185.121 75.4053,190.223 75.9959,195.37 76.5905,200.559 77.189,205.784 77.7916,211.041 78.3981,216.325 79.0085,221.629 79.6229,226.947 
 80.2412,232.273 80.8635,237.601 81.4896,242.921 82.1197,248.228 82.7536,253.513 83.3914,258.767 84.0331,263.981 84.6787,269.147 85.3281,274.256 85.9813,279.296 
 86.6384,284.258 87.2992,289.133 87.9639,293.908 88.6323,298.574 89.3046,303.121 89.9806,307.536 90.6604,311.808 91.3439,315.928 92.0311,319.884 92.7221,323.665 
 93.4168,327.261 94.1152,330.661 94.8173,333.854 95.523,336.832 96.2324,339.584 96.9455,342.102 97.6622,344.376 98.3826,346.399 99.1065,348.162 99.8341,349.659 
 100.565,350.882 101.3,351.827 102.038,352.488 102.78,352.859 103.526,352.938 104.275,352.72 105.027,352.203 105.783,351.385 106.543,350.264 107.305,348.839 
 108.072,347.111 108.842,345.079 109.615,342.745 110.392,340.11 111.172,337.176 111.956,333.946 112.743,330.422 113.533,326.609 114.327,322.51 115.124,318.13 
 115.924,313.475 116.728,308.548 117.535,303.358 118.345,297.909 119.159,292.21 119.976,286.268 120.796,280.091 121.62,273.688 122.446,267.069 123.276,260.245 
 124.109,253.226 124.945,246.025 125.785,238.655 126.627,231.129 127.473,223.464 128.322,215.674 129.174,207.778 130.029,199.793 130.887,191.739 131.749,183.636 
 132.613,175.508 133.48,167.376 134.351,159.264 135.224,151.199 136.101,143.204 136.98,135.308 137.862,127.538 138.748,119.921 139.636,112.486 140.527,105.262 
 141.422,98.2767 142.319,91.5582 143.219,85.1343 144.121,79.0321 145.027,73.2774 145.936,67.8949 146.847,62.9081 147.761,58.3385 148.678,54.2059 149.598,50.